In [1]:
import pandas as pd
import io
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from google.colab import files

## Data loading

In [2]:
# Mounting the google drive to google colab in order to load the data files directly from it
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# The data can be load directly from it
# You must change the path if the data is not directly in the path EPITA_NLP/Course1/ of the google drive
quora = pd.read_csv('/content/drive/MyDrive/EPITA_NLP/Course1/quora_questions.csv')
print(quora.head(30))
# We consider only yhe 10000 first files in order to decrease the computation time
texts = quora["Question"][0:10000]

                                             Question
0   What is the step by step guide to invest in sh...
1   What is the story of Kohinoor (Koh-i-Noor) Dia...
2   How can I increase the speed of my internet co...
3   Why am I mentally very lonely? How can I solve...
4   Which one dissolve in water quikly sugar, salt...
5   Astrology: I am a Capricorn Sun Cap moon and c...
6                                 Should I buy tiago?
7                      How can I be a good geologist?
8                     When do you use シ instead of し?
9   Motorola (company): Can I hack my Charter Moto...
10  Method to find separation of slits using fresn...
11        How do I read and find my YouTube comments?
12               What can make Physics easy to learn?
13        What was your first sexual experience like?
14  What are the laws to change your status from a...
15  What would a Trump presidency mean for current...
16                       What does manipulation mean?
17  Why do girls want to be 

## Latent Dirichlet Allocation (LDA)

Pre-processing

In [ ]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words="english")
dtm = cv.fit_transform(texts)

Use of the LDA algorithm

In [ ]:
LDA = LatentDirichletAllocation(n_components=7,random_state=42)
LDA.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=7, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [ ]:
LDA.components_

array([[ 2.76451366,  0.14311993, 10.19314347, ...,  0.1428574 ,
         0.14285746,  0.1434078 ],
       [12.44976244,  0.14294005, 11.45899252, ...,  2.14285544,
         0.14285749,  0.14285757],
       [ 1.3313155 ,  0.14285726,  5.26836191, ...,  0.14285747,
         0.14285754,  0.14293166],
       ...,
       [ 1.18617085,  0.14285725,  0.14297319, ...,  0.14285743,
         0.14285749,  6.1431953 ],
       [ 0.14296719,  2.14251102,  0.15440711, ...,  0.14285743,
         1.14306868,  1.14172508],
       [ 2.84294391,  0.14285725,  0.44971014, ...,  0.14285744,
         0.14315076,  0.14285758]])

In [ ]:
LDA.components_.shape

(7, 5168)

Let's have a look at the most representative words of each topic


In [ ]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['meaning', 'women', 'day', 'black', 'know', 'happen', 'use', 'good', 'people', 'feel', 'world', 'work', 'mean', 'like', 'does']


THE TOP 15 WORDS FOR TOPIC #1
['makes', 'ask', 'google', 'way', 'increase', 'used', 'movies', 'lose', 'question', 'questions', 'difference', 'weight', 'people', 'best', 'quora']


THE TOP 15 WORDS FOR TOPIC #2
['like', 'rid', 'different', 'body', 'human', 'help', 'instagram', 'time', 'travel', 'possible', 'iphone', 'way', 'buy', 'best', 'life']


THE TOP 15 WORDS FOR TOPIC #3
['movie', 'examples', 'exam', 'years', '2016', 'time', 'good', 'programming', 'language', 'old', 'year', 'start', 'india', 'learn', 'best']


THE TOP 15 WORDS FOR TOPIC #4
['software', 'money', 'pakistan', 'going', 'people', 'rs', 'english', 'improve', 'know', '1000', '500', 'notes', 'indian', 'think', 'india']


THE TOP 15 WORDS FOR TOPIC #5
['facebook', 'university', 'learning', 'password', 'hillary', 'clinton', 'did', 'difference', 'job', 'donald', 'goo

Associate explicitly each text to a topic

In [ ]:
topic_results_LDA = LDA.transform(dtm)
topic_results_LDA

array([[0.01786184, 0.01785901, 0.01798001, ..., 0.01787742, 0.01786066,
        0.01790007],
       [0.0476191 , 0.71415668, 0.04761911, ..., 0.0476191 , 0.04765793,
        0.04770898],
       [0.02041649, 0.20954881, 0.0204987 , ..., 0.02042326, 0.02041183,
        0.02040903],
       ...,
       [0.14705452, 0.01787374, 0.01789926, ..., 0.01786041, 0.01786808,
        0.01786091],
       [0.04775166, 0.04767432, 0.04765453, ..., 0.04763673, 0.0478086 ,
        0.04770726],
       [0.01787732, 0.01798615, 0.89241139, ..., 0.01785719, 0.01795108,
        0.01788883]])

In [ ]:
best_topic_results_LDA = topic_results_LDA.argmax(axis=1)
best_topic_results_LDA

array([3, 1, 3, ..., 3, 3, 2])

In [ ]:
df_topic_LDA = pd.DataFrame({"text":texts,"topic_LDA":best_topic_results_LDA})
df_topic_LDA

,text,topic_LDA
0,What is the step by step guide to invest in sh...,3
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,1
2,How can I increase the speed of my internet co...,3
3,Why am I mentally very lonely? How can I solve...,0
4,"Which one dissolve in water quikly sugar, salt...",6
...,...,...
9995,How would you order these four cities (Bangalo...,3
9996,Stphen william hawking?,0
9997,Mathematical Puzzles: What is () + () + () = 3...,3
9998,Is IMS noida good for BCA?,3


## Non-negative Matrix Factorization

Pre-processing

In [ ]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words="english")
dtm = tfidf.fit_transform(texts)

Use of the NMF algorithm

In [ ]:
NMF = NMF(n_components=7,random_state=42)
NMF.fit(dtm)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=7, random_state=42, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [ ]:
NMF.components_

array([[9.77045613e-03, 0.00000000e+00, 3.99174343e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.10197702e-03, 6.57957076e-04, 7.33718615e-03, ...,
        0.00000000e+00, 3.37064629e-03, 1.36034722e-02],
       [3.52025221e-02, 0.00000000e+00, 2.40301051e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 1.13241689e-03, ...,
        1.40395413e-06, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.76065709e-04, 1.54819213e-02, ...,
        5.99664451e-05, 0.00000000e+00, 6.49240909e-03],
       [7.82068526e-03, 2.78745510e-04, 1.35451534e-02, ...,
        0.00000000e+00, 0.00000000e+00, 6.94220112e-04]])

In [ ]:
NMF.components_.shape

(7, 5168)

Let's have a look at the most representative words of each topic


In [ ]:
for index,topic in enumerate(NMF.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['start', 'movie', 'weight', 'english', 'learning', 'books', 'book', 'programming', '2016', 'language', 'movies', 'india', 'learn', 'way', 'best']


THE TOP 15 WORDS FOR TOPIC #1
['sex', 'time', 'says', 'english', 'use', 'compare', 'cost', 'exist', 'love', 'india', 'long', 'feel', 'work', 'mean', 'does']


THE TOP 15 WORDS FOR TOPIC #2
['free', 'day', 'did', 'black', 'friends', 'notes', '500', 'india', '1000', 'ways', 'way', 'earn', 'online', 'money', 'make']


THE TOP 15 WORDS FOR TOPIC #3
['marked', 'people', 'post', 'earn', 'needing', 'improvement', 'asked', 'google', 'delete', 'answers', 'answer', 'ask', 'question', 'questions', 'quora']


THE TOP 15 WORDS FOR TOPIC #4
['knowledge', 'java', 'machine', 'systems', 'transgender', 'bank', 'computer', 'information', 'main', 'engineering', 'science', 'data', 'job', 'love', 'difference']


THE TOP 15 WORDS FOR TOPIC #5
['person', 'things', 'want', 'believe', 'world', 'donald', 'stop', 'don', 'did', 'trump', '

Associate explicitly each text to a topic

In [ ]:
topic_results_NMF = NMF.transform(dtm)
topic_results_NMF

array([[6.35638184e-03, 4.23533616e-03, 5.85344114e-03, ...,
        3.54505360e-04, 4.48542209e-03, 3.33288950e-03],
       [1.53374692e-03, 1.88791162e-03, 1.28095401e-03, ...,
        0.00000000e+00, 7.96641923e-04, 0.00000000e+00],
       [2.22445509e-03, 1.14807619e-03, 4.32145572e-03, ...,
        1.76950001e-04, 1.82910055e-03, 1.82246732e-03],
       ...,
       [5.08585971e-04, 7.83481024e-04, 1.38217782e-03, ...,
        1.72547696e-04, 1.95547384e-03, 1.56332490e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.87458540e-01],
       [1.90708893e-03, 2.32254332e-04, 0.00000000e+00, ...,
        8.94936362e-05, 3.98011548e-04, 4.09379511e-04]])

In [ ]:
best_topic_results_NMF = topic_results_NMF.argmax(axis=1)
best_topic_results_NMF

array([0, 1, 3, ..., 3, 6, 0])

In [ ]:
df_topic_LDA_NMF = df_topic_LDA
df_topic_LDA_NMF["topic_NMF"] = best_topic_results_NMF
df_topic_LDA_NMF

,text,topic_LDA,topic_NMF
0,What is the step by step guide to invest in sh...,3,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,1,1
2,How can I increase the speed of my internet co...,3,3
3,Why am I mentally very lonely? How can I solve...,0,1
4,"Which one dissolve in water quikly sugar, salt...",6,1
...,...,...,...
9995,How would you order these four cities (Bangalo...,3,5
9996,Stphen william hawking?,0,0
9997,Mathematical Puzzles: What is () + () + () = 3...,3,3
9998,Is IMS noida good for BCA?,3,6


Reference: https://www.udemy.com/course/nlp-natural-language-processing-with-python